- 모든 rotation을 특정해서 미리 key를 만들어두면 한방에 rotation 가능. 
- 1 bootstrapping takes ~ 10min.
- In later stages, there're up to 64

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import numpy as np
import pickle
import torch
import matplotlib.pyplot as plt 
#from fase.core import heaan

import fase
fase.USE_CUDA = False#True
#from fase import HEAAN
#from fase import HEAAN as he
from fase.core import heaan
#he = heaan.he
from typing import List, Callable
from fase.core.heaan import HEAANContext 
#from fase.HEAAN import Ciphertext

import fase.nn.utils as utils

from time import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
import torch

from fase.nn. torch_cnn import *

logI = 4, logT = 3, logq = 37, logp = 27, logn = 10, logQ = 800

In [28]:
hec = heaan.HEAANContext(10, 30, 35, boot=True, load_sk=True, rot_l="all")
#hec = heaan.HEAANContext(10, 30, 360, boot=True, load_sk=True, rot_l="all")
logqBoot = 35
hec.parms.logqBoot = logqBoot

Initializing the scheme as the data owner
Loading a secret key from:  ./SecretKey.txt
loading secret key done.
loading boot keys done
HEAAN CKKS setup is ready 


## Torch model

In [17]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

device = "cpu"

train_loader, valid_loader, test_loader = get_data_loaders()

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
        'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


## Load trained model

In [18]:
from fase.nn.models import ConvNeuralNet
from fase.nn.approximate import approx_relu, approx_sign

xfactor = 20
activation = lambda x : xfactor * approx_relu(x/xfactor, degree = 5, repeat=3)

# F.relu or activation?
org_model = ConvNeuralNet(num_classes=10, activation=F.relu) 

In [19]:
fn_param = "SimpleCNN_ReLU_minimax_v2.pt"

trained_param = torch.load(fn_param, map_location=torch.device(device))
trained_param = {k: v.cpu() for k, v in trained_param.items()} # to cpu()
org_model.load_state_dict(trained_param)
org_model.eval() ## If not eval(), running_mean and running_var of batch_norm changes

# To numpy
params_np = {k: v.numpy() for k, v in trained_param.items()}

## Load Image

In [20]:
imgs, labels = next(iter(test_loader))

# HEAAN FHE

In [21]:
# convenience tool
util = utils.Heaan_checker(hec)

In [22]:
#def do_bootstrap(fhec, ctxt):
#    ctxt = fhec.decrypt(ctxt)
#    return fhec.encrypt(ctxt)

def do_bootstrap(fhec, ctxt):
    return fhec.bootstrap(ctxt, fhec.parms.logqBoot)

In [23]:
from fase.core.commonAlgo import CommonAlgorithms
from fase.nn.conv_he import *

## step-by-step check 

In [24]:
img_this_example = imgs[0]
img_enc = [hec.encrypt(this_channel.ravel()) for this_channel in img_this_example]

print(img_enc[0].logq)

nh, nw = img_this_example[0].shape

35


t0 = time()
hec.bootstrap(img_enc[0], hec.parms.logqBoot)
print(f"took: {time() - t0:.3f}s", flush=True)

t0 = time()
hec.bootstrap(img_enc[1], hec.parms.logqBoot)
print(f"took: {time() - t0:.3f}s", flush=True)

t0 = time()
hec.bootstrap(img_enc[2], hec.parms.logqBoot)
print(f"took: {time() - t0:.3f}s", flush=True)
#[hec.bootstrap(img, hec.parms.logqBoot) for img in img_enc]
print(img_enc)

In [25]:
#for i in range(10):
if True:
    i = 0
    img_tensor = imgs[i:i+1]
    org_tt1 = org_model.conv_layer1(img_tensor)
    org_tt2 = org_model.conv_layer2(org_tt1)
    org_tt3 = org_model.pool(org_tt2)
    org_tt4 = org_model.conv_layer3(org_tt3)
    org_tt5 = org_model.bn1(org_tt4)
    org_tt6 = org_model.activation(org_tt5)
    org_tt7 = org_model.pool(org_tt6)
    org_tt8 = org_model.conv_layer4(org_tt7)
    org_tt9 = org_model.conv_layer5(org_tt8)
    #######################
    org_tt9_1 = org_model.bn2(org_tt9)
    org_tt9_2 = org_model.activation(org_tt9_1)
    #######################
    org_tt10 = org_model.pool(org_tt9_2)
    org_tt10_ = org_tt10.reshape(org_tt10.size(0),-1)
    org_11 = org_model.fc1(org_tt10_)
    org_11_1 = org_model.activation(org_11)
    answer = org_model.fc2(org_11_1)
    print(answer)
    torch.argmax(answer)

tensor([[  1.6387,   3.3497, -11.8637,  14.3675, -15.4812,   8.2658,  -8.6026,
          -7.4570,   2.9418,  11.7727]], grad_fn=<AddmmBackward0>)


In [30]:
ff = approx_sign(5)
calgo = CommonAlgorithms(hec, "HEAAN")

In [ ]:
if True:
    t0 = time()
    
    img_this_example = img_tensor[0]
    nh, nw = img_this_example[0].shape
    
    img_enc = [hec.encrypt(this_channel.ravel()) for this_channel in img_this_example]
    #[do_bootstrap(hec, img) for img in img_enc]
    logqBoot = 37
    for tt in img_enc:
        hec._scheme.bootstrapAndEqual(tt, 
                                      logqBoot, 
                                      hec.parms.logQ, 
                                      hec.parms.logT,
                                      hec.parms.logI)
        print(tt)

    

    tmp1, _nh, _nw = my_conv2D_FHE(hec, img_enc, nh, nw, org_model.conv_layer1)
    print(f"conv1... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt1, tmp1, 32,32, ch=0, unpad=0)

    tmp2, _nh2, _nw2 = my_conv2D_FHE(hec, tmp1, nh, nw, org_model.conv_layer2) # list of ctxts
    print(f"conv2... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt2, tmp2, 32,32, ch=0, unpad=0)

    tmp3, nh2, nw2 = fhe_avg_pool(hec, tmp2,
                                  nh, nw, 
                                  kernel_size=org_model.pool.kernel_size, 
                                  stride_in=1) # list of ctxts
    print(f"pool1... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt3, tmp3, 32,32, ch=5, unpad=0, stride=(2,2))

    tmp4, _nh2, _nw2 = my_conv2D_FHE(hec, tmp3, nh, nw, org_model.conv_layer3,
                                    stride_in=2) # list of ctxts
    print(f"conv3... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt4, tmp4, 32,32, ch=5, unpad=0, stride=(2,2))

    tmp5 = fhe_bn(hec, tmp4, org_model.bn1, eps = 1e-5)
    print(f"bn1... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt5, tmp5, 32,32, ch=5, unpad=0, stride=(2,2))

    ## Bootstrap?
    print("1", tmp5[0].logp, tmp5[0].logq)
   
    [do_bootstrap(hec, tmp) for tmp in tmp5]

    tmp6 = approx_relu_fhe(hec, calgo, tmp5, ff, repeat=4, xfactor=20)
    print(f"activation... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt6, tmp6, 32,32, ch=5, unpad=0, stride=(2,2))

    #tmp6 = [do_bootstrap(hec, tmp) for tmp in tmp6]
    ## Bootstrap?
    print("2", tmp6[0].logp, tmp6[0].logq)

    tmp7, _nh3, _nw3 = fhe_avg_pool(hec, tmp6, nh, nw, 
                                    kernel_size=org_model.pool.kernel_size, 
                                    stride_in=2) # list of ctxts
    print(f"activation... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt7, tmp7, 32,32, ch=5, unpad=0, stride=(4,4))

    tmp8, _nh3, _nw3 = my_conv2D_FHE(hec, tmp7, nh, nw, org_model.conv_layer4,
                                    stride_in=4) # list of ctxts
    print(f"conv4... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt8, tmp8, 32,32, ch=5, unpad=0, stride=(4,4))
    
    tmp9, _nh3, _nw3 = my_conv2D_FHE(hec, tmp8, nh, nw, org_model.conv_layer5,
                                    stride_in=4) # list of ctxts
    print(f"conv5... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt9, tmp9, 32,32, ch=5, unpad=0, stride=(4,4))
    
    tmp9_1 = fhe_bn(hec, tmp9, org_model.bn2, eps = 1e-5) #fhe_bn(hec, tmp9, org_model.bn2)
    print(f"bn2... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt9_1, tmp9_1, 32,32, ch=5, unpad=0, stride=(4,4))
    
    ## Bootstrap?
    print("3", tmp9_1[0].logp, tmp9_1[0].logq)
    
    tmp9_2 = approx_relu_fhe(hec, calgo, tmp9_1, ff, repeat=4, xfactor=20)
    print(f"activation.. {time() - t0:.3f} seconds")
    util.plot_diff(org_tt9_2, tmp9_2, 32,32, ch=5, unpad=0, stride=(4,4))
    
    
    tmp10, _nh4, _nw4 = fhe_avg_pool(hec, tmp9_2, nh, nw, 
                                    kernel_size=org_model.pool.kernel_size, 
                                    stride_in=4) # list of ctxts
    print(f"pool2... {time() - t0:.3f} seconds")
    util.plot_diff(org_tt10, tmp10, 32,32, ch=5, unpad=0, stride=(8,8))
    
    reshaped = reshape(hec, tmp10)

    print("4", reshaped[0].logp, reshaped[0].logq)
    reshaped = do_bootstrap(hec, reshaped)

    tmp_fc1 = fullyConnected(hec, reshaped, org_model.fc1)
    print(f"FC1... {time() - t0:.3f} seconds")

    #tmp_fc1 = [do_bootstrap(hec, tmp_fc1)]
    tmp_fc11 = approx_relu_fhe(hec, calgo, tmp_fc1, ff, repeat=4, xfactor=15)[0]
    print(f"activation... {time() - t0:.3f} seconds")

    result = fullyConnected(hec, tmp_fc11, org_model.fc2)
    print(f"Done... {time() - t0:.3f} seconds")

    pred = hec.decrypt(result)[:10]
    
    pred = fhe_eval(org_model, sec, img_tensor[0])
    print(pred)
    print("_________\n")

In [ ]:
def fhe_eval(org_model, hec, img_this_example):
    t0 = time()

    img_enc = [hec.encrypt(this_channel.ravel()) for this_channel in img_this_example]
    nh, nw = img_this_example[0].shape

    tmp1, _nh, _nw = my_conv2D_FHE(hec, img_enc, nh, nw, org_model.conv_layer1)
    print(f"conv1... {time() - t0:.3f} seconds")

    tmp2, _nh2, _nw2 = my_conv2D_FHE(hec, tmp1, nh, nw, org_model.conv_layer2) # list of ctxts
    print(f"conv2... {time() - t0:.3f} seconds")

    tmp3, nh2, nw2 = fhe_avg_pool(hec, tmp2,
                                  nh, nw, 
                                  kernel_size=org_model.pool.kernel_size, 
                                  stride_in=1) # list of ctxts
    print(f"pool1... {time() - t0:.3f} seconds")

    tmp4, _nh2, _nw2 = my_conv2D_FHE(hec, tmp3, nh, nw, org_model.conv_layer3,
                                    stride_in=2) # list of ctxts
    print(f"conv3... {time() - t0:.3f} seconds")

    #tmp5 = fhe_bn(hec, tmp4, org_model.bn1)
    tmp5 = fhe_bn(hec, tmp4, org_model.bn1, eps = 1e-5)
    print(f"bn1... {time() - t0:.3f} seconds")

    print("1", tmp5[0].logp, tmp5[0].logq)

    tmp5 = [do_bootstrap(hec, tmp) for tmp in tmp5]

    tmp6 = approx_relu_fhe(hec, calgo, tmp5, ff, repeat=4, xfactor=20)
    print(f"activation... {time() - t0:.3f} seconds")

    #tmp6 = [do_bootstrap(hec, tmp) for tmp in tmp6]
    print("2", tmp6[0].logp, tmp6[0].logq)

    tmp7, _nh3, _nw3 = fhe_avg_pool(hec, tmp6, nh, nw, 
                                    kernel_size=org_model.pool.kernel_size, 
                                    stride_in=2) # list of ctxts
    print(f"activation... {time() - t0:.3f} seconds")

    tmp8, _nh3, _nw3 = my_conv2D_FHE(hec, tmp7, nh, nw, org_model.conv_layer4,
                                    stride_in=4) # list of ctxts
    print(f"conv4... {time() - t0:.3f} seconds")
    tmp9, _nh3, _nw3 = my_conv2D_FHE(hec, tmp8, nh, nw, org_model.conv_layer5,
                                    stride_in=4) # list of ctxts
    print(f"conv5... {time() - t0:.3f} seconds")
    tmp9_1 = fhe_bn(hec, tmp9, org_model.bn2, eps = 1e-5) #fhe_bn(hec, tmp9, org_model.bn2)

    print("3", tmp9_1[0].logp, tmp9_1[0].logq)

    print(f"bn2... {time() - t0:.3f} seconds")
    tmp9_2 = approx_relu_fhe(hec, calgo, tmp9_1, ff, repeat=4, xfactor=20)
    print(f"activation.. {time() - t0:.3f} seconds")
    tmp10, _nh4, _nw4 = fhe_avg_pool(hec, tmp9_2, nh, nw, 
                                    kernel_size=org_model.pool.kernel_size, 
                                    stride_in=4) # list of ctxts
    print(f"pool2... {time() - t0:.3f} seconds")
    reshaped = reshape(hec, tmp10)

    print("4", reshaped[0].logp, reshaped[0].logq)
    reshaped = do_bootstrap(hec, reshaped)

    tmp_fc1 = fullyConnected(hec, reshaped, org_model.fc1)
    print(f"FC1... {time() - t0:.3f} seconds")

    #tmp_fc1 = [do_bootstrap(hec, tmp_fc1)]
    tmp_fc11 = approx_relu_fhe(hec, calgo, tmp_fc1, ff, repeat=4, xfactor=15)[0]
    print(f"activation... {time() - t0:.3f} seconds")

    result = fullyConnected(hec, tmp_fc11, org_model.fc2)
    print(f"Done... {time() - t0:.3f} seconds")

    pred = hec.decrypt(result)[:10]
    
    return pred